In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from ipywidgets import interact, FloatSlider, Label
from shapely.geometry import Point
from shapely.plotting import plot_polygon

# Exercise 4: **Frequency Modulated Radar**

*Visualize and explain the operation of a frequency-modulated radar.*

## *How does a FM-Radar (in principle) work?*

The frequency modulated Radar belongs to the family of ranging radars and side looking radars. The basic idea is, that instead of sending a stream of temporally limited pulses, a continuous signal is sent, which gets modulated in its freuency with a given modulation pattern. One such modualtion could be a triangular wave pattern, according to which the frequencies are changed contoniusly. 

$f\left(t, f_1, f_2\right) = \dfrac{\left|f_1 - f_2\right|}{2} \cdot \dfrac{2}{\pi} \arcsin\left(\sin\left(2\pi f_{T} t\right)\right) + f_1 \dfrac{\left|f_1 - f_2\right|}{2}$

where $t$ is the time, $f_1$ and $f_2$ the cutoff frequencies of the bandwidth and $f_T$ the frequency of the triangular wave (sweep time $t_R = f_T^{-1}$).

Therefore,

$f\left(t\right) = \dfrac{B}{\pi} \arcsin\left(\sin\left(2\pi f_{T} t\right)\right) + f_1 \dfrac{B}{2}$


The echo will return after a time $\tilde{t} = \dfrac{2R}{v}$, depending on the propagation velocity $v$ and the distance between satellite sensor and target $R$. When a signal is received, because the modualtion of the transmitted signal is known, comparing the frequency offset between both can be related to the distance the signal propagated. 

The frequency offset between transmitted and received signal $\Delta f$ is given as (assuming the signal started transmitting at $t=0$ and that the echo arrives within the first quarter of the triangular waveform):

$\Delta f = f\left(\tilde{t}\right) - f\left(t=0\right) = 2B\dfrac{\tilde{t}}{t_R} = \dfrac{4BR}{t_Rv}$

Thus, the distance $R$ can be expressed as a function of the frequency offset betwenn transmitted and received signal $\Delta f$:

$R = \dfrac{t_{R}v}{4B}\cdot \Delta f$


*Note: Even if not mentioned explicitly in the lecture, the maximum distance detectable reliably depends on the modulation pattern and if a single pattern is used repeatadly or not. As the modulation pattern consists of a upwards and downwards sweep of frequencies, most frequencies are visited twice. Independent of the shape of the modulation. Thus, when comparing the received signal to a transmited one, at least two points in time of both signals have to be sampled (to determine if one is within the up- or downsweep). The approach presented above is thus only valid for some cases and does not hold true in general.*

## *What is the main quantity governing the resolution of a FM-Radar?*
The bandwidth of the transmitted signal determines the resolution obtainable. Though, underlying technical limitations are probably even more important. One such limitation could be the sensors ability to change between two frequencies during the sweeping. If this changig time it too long, a drop in resolution is to be expected.

## *How does an FM-Radar diﬀer from an ordinary Pulse-Radar?*

An ordinary pulse Radar transmits pulses at such an offset in time, that in between two pulses the echo can be received. The FM-Radar transmits continously. 


In [2]:
F1 = 0.3    #GHz
F2 = 1      #GHz
R = 800     #km

In [3]:
def triangular_pattern(frequency, f1, f2, num_cycles, num_samples = 1000, delay_time = 0):
    amplitude = (f2 - f1) / 2
    t = np.linspace(0, num_cycles * 2 * np.pi / frequency, num_samples, endpoint=False)
    triangular_wave = amplitude * (2 / np.pi) * np.arcsin(np.sin(2 * np.pi * frequency * t)) + (f1 + amplitude)
    padding = [np.nan for dt in range(delay_time)]
    triangular_wave = padding + list(triangular_wave)
    return t, triangular_wave[ : len(t)]

In [4]:
f1_slider = FloatSlider(value = F1, min = 0.01, max = 0.5, step = 0.01, description = 'f_1 [GHz]:')
f2_slider = FloatSlider(value=F2, min=0.6, max=1.5, step=0.1, description = 'f_2 [GHz]:')
r_slider = FloatSlider(value=23, min=0, max=70, step=1, description='R [a.u.]:')
f_sweep = FloatSlider(value=2, min=1, max=4, step=0.1, description='sweep frequency [a.u.]:')

@interact(f1 = f1_slider, f2 = f2_slider, r = r_slider, frequency = f_sweep)
def update(f1, f2, r, frequency):
    fig = plt.figure(figsize = (14, 6))
    ax0 = fig.add_subplot(111)
    tt, transmitted = triangular_pattern(frequency=frequency, f1 =f1, f2 =f2, num_cycles=0.5)
    tr, received = triangular_pattern(frequency =frequency, f1 = f1, f2 =f2, num_cycles=0.5, delay_time = int(r))
    ax0.plot(tt, transmitted, linewidth = 2, color = 'black', label = 'transmitted signal')
    ax0.plot(tr, received, linewidth = 2, color = 'red', label = 'received signal')
    ax0.set_ylim(0.01, 1.5)
    ax0.set_xlim(-0.03, 0.7)

    ax0.set_ylabel('Frequency [GHz]', fontsize = 12)
    ax0.set_xlabel('Time [a.u.]', fontsize = 12)

    delta_f = np.abs(transmitted[int(r)] - received[int(r)])
    ax0.set_title(f'B={np.abs(f1-f2):.2f}GHz, f1 = {f1:.2f}GHz, f2 = {f2:.2f}GHz, T_R = {(1/frequency):.2f}a.u.,\nR = {r:.2f}[a.u.] corresponding to delta_f = {delta_f:.2f}GHz', fontsize = 16)
    
    ax0.plot([tt[int(r)], tt[int(r)]], [transmitted[int(r)], received[int(r)]], linestyle = 'dotted', linewidth = 2, color = 'blue', label = r'$\Delta f = $' + f'{delta_f:.2f}GHz')


    plt.legend()



interactive(children=(FloatSlider(value=0.3, description='f_1 [GHz]:', max=0.5, min=0.01, step=0.01), FloatSli…